In [1]:
from src.search_engine import SearchEngine
from src.model import Model
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [30]:
def get_best_using_sparse(query, texts):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(texts)
    query_sparse = vectorizer.transform([query]).toarray()
    knn = NearestNeighbors(algorithm="brute").fit(X.toarray())
    best_text_idx = knn.kneighbors(query_sparse, 1)[1][0, :][0]
    return texts[best_text_idx]

In [3]:
preprocessed_data_dir = "data/preprocessed_corpus.json"
embeddings_dir = "data/corpus_embeddings.json"

In [31]:
model = Model("distiluse-base-multilingual-cased-v1")
search = SearchEngine(embeddings_dir, preprocessed_data_dir, model)

100848

In [29]:
known_definitions = [text.split(".")[0] for text in search.text]
len(known_definitions)

100848

In [32]:
all_questions = []
with open("data/def_question.tsv", encoding="utf-8") as f:
    for line in f.readlines():
        line = line.strip().split("\t")
        question, answer = line[0], line[1:]
        all_questions.append((question, answer))
        
len(all_questions)

248

In [37]:
dense_correct = 0
dense_sparse_correct = 0
for query, answers in all_questions:
    print("Query: {}, Answers: {}".format(query, answers))
    dense_result = search.run(query)[0]
    result_word = dense_result.split(".")[0]
    dense_correct += int(result_word in answers)
    print("Best text using dense rep: \n\t{}".format(dense_result), end="")
    candidates = search.run(query, k_best=20)
    sparse_result = get_best_using_sparse(query, candidates)
    result_word = sparse_result.split(".")[0]
    dense_sparse_correct += int(result_word in answers)
    print("Best text using dense and sparse rep:\n\t{}".format(sparse_result))

Query: Jak nazywa się pierwsza litera alfabetu greckiego?, Answers: ['alfa']
Best text using dense rep: 
	alfa. Definicja:  {{jęz}} nazwa pierwszej litery alfabetu greckiego, α; {{wikipedia}}
Best text using dense and sparse rep:
	pi. Definicja:  nazwa szesnastej litery alfabetu greckiego, 

Query: Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?, Answers: ['cięciwa']
Best text using dense rep: 
	apsyda. Definicja:  {{astr}} jeden z dwóch skrajnych punktów orbity eliptycznej;
Best text using dense and sparse rep:
	dwurzędowy. Definicja:  ''(o ubraniu)'' mający dwa rzędy guzików

Query: Jak nazywał się sztywny kapelusz męski o zaokrąglonej główce i wąskim lekko uniesionym rondelku, modny w drugiej połowie XIX wieku?, Answers: ['melonik']
Best text using dense rep: 
	maciejówka. Definicja:  {{kraw}} męska czapka z okrągłym denkiem wykonana z sukna;
Best text using dense and sparse rep:
	melonik. Definicja:  sztywny, półkolisty kapelusz męski

Query: Jak nazywa się trucizna przyr

In [38]:
print("DENSE:\n Correct answers: {} ({:.2f}%)".format(dense_correct, dense_correct * 100/ len(all_questions)))
print("DENSE + SPARSE:\n Correct answers: {} ({:.2f}%)".format(dense_sparse_correct, dense_sparse_correct * 100/ len(all_questions)))

DENSE:
 Correct answers: 77 (31.05%)
DENSE + SPARSE:
 Correct answers: 43 (17.34%)
